In [1]:
from __future__ import division
import argparse

from PIL import Image
import numpy as np
import gym

from keras.models import Sequential
from keras.layers import Dense, Activation, Flatten, Convolution2D, Permute
from keras.optimizers import Adam
import keras.backend as K

from rl.agents.dqn import DQNAgent
from rl.policy import LinearAnnealedPolicy, BoltzmannQPolicy, EpsGreedyQPolicy
from rl.memory import SequentialMemory
from rl.core import Processor
from rl.callbacks import FileLogger, ModelIntervalCheckpoint

Using TensorFlow backend.


In [2]:
INPUT_SHAPE = (84, 84)
WINDOW_LENGTH = 4


class AtariProcessor(Processor):
    def process_observation(self, observation):
        assert observation.ndim == 3  # (height, width, channel)
        img = Image.fromarray(observation)
        img = img.resize(INPUT_SHAPE).convert('L')  # resize and convert to grayscale
        processed_observation = np.array(img)
        assert processed_observation.shape == INPUT_SHAPE
        return processed_observation.astype('uint8')  # saves storage in experience memory

    def process_state_batch(self, batch):
        # We could perform this processing step in `process_observation`. In this case, however,
        # we would need to store a `float32` array instead, which is 4x more memory intensive than
        # an `uint8` array. This matters if we store 1M observations.
        processed_batch = batch.astype('float32') / 255.
        return processed_batch

    def process_reward(self, reward):
        return np.clip(reward, -1., 1.)

In [3]:
game = 'Breakout-v4'

In [4]:
# Get the environment and extract the number of actions.
env = gym.make(game)
np.random.seed(123)
env.seed(123)
nb_actions = env.action_space.n

# Next, we build our model. We use the same model that was described by Mnih et al. (2015).
input_shape = (WINDOW_LENGTH,) + INPUT_SHAPE
model = Sequential()

# (width, height, channels)
model.add(Permute((2, 3, 1), input_shape=input_shape))
model.add(Convolution2D(32, (8, 8), strides=(4, 4)))
model.add(Activation('relu'))
model.add(Convolution2D(64, (4, 4), strides=(2, 2)))
model.add(Activation('relu'))
model.add(Convolution2D(64, (3, 3), strides=(1, 1)))
model.add(Activation('relu'))
model.add(Flatten())
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dense(nb_actions))
model.add(Activation('linear'))
print(model.summary())


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
permute_1 (Permute)          (None, 84, 84, 4)         0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 20, 20, 32)        8224      
_________________________________________________________________
activation_1 (Activation)    (None, 20, 20, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 9, 9, 64)          32832     
_________________________________________________________________
activation_2 (Activation)    (None, 9, 9, 64)          0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 7, 7, 64)          36928     
_________________________________________________________________
activation_3 (Activation)    (None, 7, 7, 64)          0         
__________

In [5]:
# Finally, we configure and compile our agent. You can use every built-in Keras optimizer and
# even the metrics!
memory = SequentialMemory(limit=1000000, window_length=WINDOW_LENGTH)
processor = AtariProcessor()

# Select a policy. We use eps-greedy action selection, which means that a random action is selected
# with probability eps. We anneal eps from 1.0 to 0.1 over the course of 1M steps. This is done so that
# the agent initially explores the environment (high eps) and then gradually sticks to what it knows
# (low eps). We also set a dedicated eps value that is used during testing. Note that we set it to 0.05
# so that the agent still performs some random actions. This ensures that the agent cannot get stuck.
policy = LinearAnnealedPolicy(EpsGreedyQPolicy(), attr='eps', value_max=1., value_min=.1, value_test=.05,
                              nb_steps=1000000)

# The trade-off between exploration and exploitation is difficult and an on-going research topic.
# If you want, you can experiment with the parameters or use a different policy. Another popular one
# is Boltzmann-style exploration:
# policy = BoltzmannQPolicy(tau=1.)
# Feel free to give it a try!

dqn = DQNAgent(model=model, nb_actions=nb_actions, policy=policy, memory=memory,
               enable_dueling_network=False, dueling_type='avg',
               processor=processor, nb_steps_warmup=50000, gamma=.99, target_model_update=10000,
               train_interval=4, delta_clip=1.)
dqn.compile(Adam(lr=.00025), metrics=['mae'])

In [ ]:
weights_filename = 'DQN/weights/dqn_{}_weights.h5f'.format(game)
checkpoint_weights_filename = 'DQN/weights/dqn_' + game + '_weights_{step}.h5f'
log_filename = 'DQN/logs/dqn_{}_log.json'.format(game)
callbacks = [ModelIntervalCheckpoint(checkpoint_weights_filename, interval=250000)]
callbacks += [FileLogger(log_filename, interval=100)]
dqn.fit(env, callbacks=callbacks, nb_steps=10000000, log_interval=10000)

Training for 10000000 steps ...
Interval 1 (0 steps performed)
10000/10000 [==============================] - 20s 2ms/step - reward: 0.0058
39 episodes - episode_reward: 1.436 [0.000, 5.000] - ale.lives: 2.962

Interval 2 (10000 steps performed)
10000/10000 [==============================] - 19s 2ms/step - reward: 0.0059
39 episodes - episode_reward: 1.564 [0.000, 6.000] - ale.lives: 3.021

Interval 3 (20000 steps performed)
10000/10000 [==============================] - 19s 2ms/step - reward: 0.0058
39 episodes - episode_reward: 1.436 [0.000, 5.000] - ale.lives: 3.050

Interval 4 (30000 steps performed)
10000/10000 [==============================] - 19s 2ms/step - reward: 0.0061
38 episodes - episode_reward: 1.605 [0.000, 7.000] - ale.lives: 2.938

Interval 5 (40000 steps performed)
10000/10000 [==============================] - 19s 2ms/step - reward: 0.0061
39 episodes - episode_reward: 1.615 [0.000, 4.000] - ale.lives: 2.919

Interval 6 (50000 steps performed)
10000/10000 [=========

In [ ]:
env = gym.make(game)
dqn.load_weights("weights/dqn_Breakout-v4_weights_1750000.h5f")
dqn.test(env, nb_episodes=10, visualize=True)

Testing for 10 episodes ...
Episode 1: reward: 39.000, steps: 1950
Episode 2: reward: 48.000, steps: 2083
Episode 3: reward: 40.000, steps: 1897
Episode 4: reward: 48.000, steps: 2299
Episode 5: reward: 35.000, steps: 1590
Episode 6: reward: 29.000, steps: 1450
Episode 7: reward: 40.000, steps: 1909
Episode 8: reward: 37.000, steps: 1806
